<a href="https://colab.research.google.com/github/sayak-coding/Stock-Market-Analysis-Prediction/blob/main/Stock_Market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pandas Library
import pandas as pd
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# # yfinance is used to fetch data
# import yfinance as yf
# yf.pdr_override()

In [ ]:
# # input
# symbol = 'AMZN'
# start = '2014-01-01'
# end = '2023-12-31'

# # Read data
# dataset = yf.download(symbol,start,end)

# # View Columns
# # Dataframe
# dataset.head()

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/AMZN.csv")

In [ ]:
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-01-02,19.940001,19.9680,19.701000,19.898500,19.898500,42756000
1,2014-01-03,19.914499,20.1355,19.811001,19.822001,19.822001,44204000
2,2014-01-06,19.792500,19.8500,19.421000,19.681499,19.681499,63412000
3,2014-01-07,19.752001,19.9235,19.714500,19.901501,19.901501,38320000
4,2014-01-08,19.923500,20.1500,19.802000,20.096001,20.096001,46330000


In [ ]:
dataset.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
2511,2023-12-22,153.770004,154.350006,152.710007,153.419998,153.419998,29480100
2512,2023-12-26,153.559998,153.979996,153.029999,153.410004,153.410004,25067200
2513,2023-12-27,153.559998,154.779999,153.119995,153.339996,153.339996,31434700
2514,2023-12-28,153.720001,154.080002,152.949997,153.380005,153.380005,27057000
2515,2023-12-29,153.100006,153.889999,151.029999,151.940002,151.940002,39789000


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2516 non-null   object 
 1   Open       2516 non-null   float64
 2   High       2516 non-null   float64
 3   Low        2516 non-null   float64
 4   Close      2516 non-null   float64
 5   Adj Close  2516 non-null   float64
 6   Volume     2516 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 137.7+ KB


In [ ]:
import datetime as dt

In [ ]:
# convert date field from string to Date format and make it index
dataset['Date'] = pd.to_datetime(dataset.Date)
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-01-02,19.940001,19.9680,19.701000,19.898500,19.898500,42756000
1,2014-01-03,19.914499,20.1355,19.811001,19.822001,19.822001,44204000
2,2014-01-06,19.792500,19.8500,19.421000,19.681499,19.681499,63412000
3,2014-01-07,19.752001,19.9235,19.714500,19.901501,19.901501,38320000
4,2014-01-08,19.923500,20.1500,19.802000,20.096001,20.096001,46330000


In [ ]:
print("Starting date: ",dataset.iloc[0][0])
print("Ending date: ", dataset.iloc[-1][0])
print("Duration: ", dataset.iloc[-1][0]-dataset.iloc[0][0])

Starting date:  2014-01-02 00:00:00
Ending date:  2023-12-29 00:00:00
Duration:  3648 days 00:00:00


In [ ]:
closedf = dataset[['Date','Close']].copy()
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (2516, 2)


In [ ]:
# Execute this cell to investigate the issue.
print(closedf.columns)

Index(['Date', 'Close'], dtype='object')


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

# ! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
close_stock = closedf.copy()
del closedf['Date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(2516, 1)


In [ ]:
training_size=int(len(closedf)*0.8)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (2012, 1)
test_data:  (504, 1)


In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (1996, 15)
y_train:  (1996,)
X_test:  (488, 15)
y_test (488,)


In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (1996, 15, 1)
X_test:  (488, 15, 1)


In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 21025 (82.13 KB)
Trainable params: 21025 (82.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=5,verbose=1)

Epoch 1/200
400/400 [==============================] - 11s 11ms/step - loss: 0.0066 - val_loss: 0.0037
Epoch 2/200
400/400 [==============================] - 4s 10ms/step - loss: 9.2918e-04 - val_loss: 0.0029
Epoch 3/200
400/400 [==============================] - 4s 10ms/step - loss: 6.8313e-04 - val_loss: 0.0024
Epoch 4/200
400/400 [==============================] - 4s 9ms/step - loss: 8.8711e-04 - val_loss: 0.0035
Epoch 5/200
400/400 [==============================] - 4s 11ms/step - loss: 6.4817e-04 - val_loss: 0.0021
Epoch 6/200
400/400 [==============================] - 3s 9ms/step - loss: 6.1021e-04 - val_loss: 0.0058
Epoch 7/200
400/400 [==============================] - 3s 9ms/step - loss: 8.1033e-04 - val_loss: 0.0033
Epoch 8/200
400/400 [==============================] - 4s 9ms/step - loss: 5.5196e-04 - val_loss: 0.0018
Epoch 9/200
400/400 [==============================] - 4s 10ms/step - loss: 4.6664e-04 - val_loss: 0.0014
Epoch 10/200
400/400 [==============================]

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

16/16 [==============================] - 0s 3ms/step


((1996, 1), (488, 1))

In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

In [ ]:
import math

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.272151168422767
Train data MSE:  5.162670932164945
Test data MAE:  1.4380623034271365
-------------------------------------------------------------------------------------
Test data RMSE:  4.6168541821254925
Test data MSE:  21.31534253900965
Test data MAE:  3.6338538277227683


In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9984497378438976
Test data explained variance regression score: 0.9716657212018467


In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9980779109266392
Test data R2 score: 0.9514904327970721


In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0005796666101176328
Test data MGD:  0.0013131504881660873
----------------------------------------------------------------------
Train data MPD:  0.044745919674853425
Test data MPD:  0.16528952883366357


In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'Date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['Date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','Date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (2516, 1)
Test predicted data:  (2516, 1)


In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(lstmdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()